# 1. Import packages

In [1]:
import glob
import json
import os
import pandas as pd

# 2. Read a txt file with URL analysis

## 2.1. Change to inputs directory

In [3]:
print(os.getcwd())
input_path_s = "/inputs"
input_path_s = os.getcwd() + input_path_s
output_path_s = "/outputs"
output_path_s = os.getcwd() + output_path_s
os.chdir(input_path_s)
print(os.getcwd())

/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task12_AS_from_task06
/Users/immanuel/Documents/NEU/05_Spring_2022/CS7675_ResAppr/tasks/MyLab/task12_AS_from_task06/inputs


## 2.2. Read all txt files

In [4]:
txt_filename_l = []
df_l = []

for file in glob.glob("*.txt"):
    # Get the txt filename without the extension
    txt_filename = os.path.splitext(file)[0][4:]
    # If there is a number at the end of the filename, then remove.
    if txt_filename[-1].isdigit():
        txt_filename = txt_filename[:-1]
    txt_filename_l.append(txt_filename)
    df_l.append(pd.read_csv(file, sep=":", header=None))
    
print(len(txt_filename_l))
print(len(df_l))

13
13


## 2.3. eu_country_d

In [5]:
eu_country_d = {'at': 'Austria'
                ,'be': 'Belgium'
                ,'bg': 'Bulgaria'
                ,'hr': 'Croatia'
                ,'cy': 'Cyprus'
                ,'cz': 'Czech'
                ,'dk': 'Denmark'
                ,'ee': 'Estonia'
                ,'fi': 'Finland'
                ,'fr': 'France'
                ,'de': 'Germany'
                ,'gr': 'Greece'
                ,'hu': 'Hungary'
                ,'ie': 'Ireland'
                ,'it': 'Italy'
                ,'lv': 'Latvia'
                ,'lt': 'Lithuania'
                ,'lu': 'Luxembourg'
                ,'mt': 'Malta'
                ,'nl': 'Netherlands'
                ,'pl': 'Poland'
                ,'pt': 'Portugal'
                ,'ro': 'Romania'
                ,'sk': 'Slovakia'
                ,'si': 'Slovenia'
                ,'es': 'Spain'
                ,'se': 'Sweden'
#                ,'gb': ''
#                ,'is': 'Iceland'
#                ,'li': 'Liechtenstein'
#                ,'no': 'Norway'
#                ,'ch': 'Switzerland'
               }
print(len(eu_country_d))

27


In [6]:
eu_country_d = dict([(value.lower(), key) for key, value in eu_country_d.items()])
print(eu_country_d)

{'austria': 'at', 'belgium': 'be', 'bulgaria': 'bg', 'croatia': 'hr', 'cyprus': 'cy', 'czech': 'cz', 'denmark': 'dk', 'estonia': 'ee', 'finland': 'fi', 'france': 'fr', 'germany': 'de', 'greece': 'gr', 'hungary': 'hu', 'ireland': 'ie', 'italy': 'it', 'latvia': 'lv', 'lithuania': 'lt', 'luxembourg': 'lu', 'malta': 'mt', 'netherlands': 'nl', 'poland': 'pl', 'portugal': 'pt', 'romania': 'ro', 'slovakia': 'sk', 'slovenia': 'si', 'spain': 'es', 'sweden': 'se'}


## 2.4. Create df names based on txt names (Assign df into country_df)

In [7]:
df_name_l = []

for i in range(len(txt_filename_l)):
    df_name = eu_country_d[txt_filename_l[i]] + "_df"
    df_name_l.append(df_name)
    locals()[df_name] = df_l[i]
    
print(len(df_name_l))
print(df_name_l)

13
['gr_df', 'it_df', 'se_df', 'ro_df', 'fr_df', 'pl_df', 'es_df', 'nl_df', 'at_df', 'cz_df', 'de_df', 'sk_df', 'hu_df']


# 3. Parse to get IPs

In [9]:
ip_d = {}

for index, country_df in enumerate(df_l):
    ip_ser = country_df.loc[:, 2]
    
    ip_l = []
    
    country_code_s = eu_country_d[txt_filename_l[index]]

    for ip_column in ip_ser:
        ip_l.append(ip_column.split("-")[-1])

    # Assign country code as a dictionary key
    # Assign ip list as a dictionary value
    country_code_s = eu_country_d[txt_filename_l[index]]
    ip_d[country_code_s] = ip_l

    print(country_code_s)
    print(len(ip_l))
    print({True: "No Dupe", False: "Yes Dupe"}[len(ip_l) == len(set(ip_l))])
    print()

print(len(ip_d))

gr
160
No Dupe

it
1002
No Dupe

se
384
No Dupe

ro
229
No Dupe

fr
2494
No Dupe

pl
596
No Dupe

es
2307
No Dupe

nl
914
No Dupe

at
217
No Dupe

cz
494
No Dupe

de
1002
No Dupe

sk
327
No Dupe

hu
208
No Dupe

13


# 4. IP to AS Name and AS Number

In [12]:
def get_descriptive_statistics_about_as_name_and_as_number(as_name_l, as_number_l):
    print(f"The length of AS Name is {len(as_name_l)}")
        
    print(f"The length of AS Number is {len(as_number_l)}")
    
    if len(as_name_l) == len(as_number_l):
        print("AS Name and AS Number have the same length.")
    elif len(as_name_l) > len(as_number_l):
        print("AS Name has a longer length than AS Number.")
    else:
        print("AS Number has a longer length than AS Name.")
   
    if len(as_name_l) == len(set(as_name_l)):
        print("No duplicates in AS Name.")
    else:
        print(f"Duplicates in AS Name. The length of AS Name without duplicates is {len(set(as_name_l))}")

    if len(as_number_l) == len(set(as_number_l)):
        print("No duplicates in AS Number.")
    else:
        print(f"Duplicates in AS Number. The length of AS Number without duplicates is {len(set(as_number_l))}")

In [13]:
as_d = {}
for key in ip_d.keys():
    as_d[key] = {}

In [14]:
for country_code, ip_l in ip_d.items():
    print(country_code)

    as_number_l = []
    as_name_l = []

    for ip in ip_l:
        cmd = f"whois -h whois.cymru.com {ip}"
        try:
            cymru_l = os.popen(cmd).read().split("\n")[1].split("|")
        except:
            print(cmd)
            break

        for index, item in enumerate(cymru_l):
            if index == 0:
                as_number_l.append(item.strip())
            if index == 2:
                as_name_l.append(item.strip())
                
    unique_as_number_l = list(set(as_number_l))
    unique_as_name_l = list(set(as_name_l))
    
    as_d[country_code]["unique_as_number"] = unique_as_number_l
    as_d[country_code]["unique_as_name"] = unique_as_name_l

    get_descriptive_statistics_about_as_name_and_as_number(as_name_l, as_number_l)
    print()

gr
The length of AS Name is 160
The length of AS Number is 160
AS Name and AS Number have the same length.
Duplicates in AS Name. The length of AS Name without duplicates is 1
Duplicates in AS Number. The length of AS Number without duplicates is 1

it
The length of AS Name is 1002
The length of AS Number is 1002
AS Name and AS Number have the same length.
Duplicates in AS Name. The length of AS Name without duplicates is 1
Duplicates in AS Number. The length of AS Number without duplicates is 1

se
The length of AS Name is 384
The length of AS Number is 384
AS Name and AS Number have the same length.
Duplicates in AS Name. The length of AS Name without duplicates is 1
Duplicates in AS Number. The length of AS Number without duplicates is 1

ro
The length of AS Name is 229
The length of AS Number is 229
AS Name and AS Number have the same length.
Duplicates in AS Name. The length of AS Name without duplicates is 1
Duplicates in AS Number. The length of AS Number without duplicates is 1

# 5. JSON

In [15]:
as_d

{'gr': {'unique_as_number': ['11325'], 'unique_as_name': ['AS11325, US']},
 'it': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'se': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'ro': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'fr': {'unique_as_number': ['209372', '9009'],
  'unique_as_name': ['WSTELECOM_CUSTOMERS, LV', 'M247, GB']},
 'pl': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'es': {'unique_as_number': [], 'unique_as_name': []},
 'nl': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'at': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'cz': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']},
 'de': {'unique_as_number': ['209372', '9009'],
  'unique_as_name': ['WSTELECOM_CUSTOMERS, LV', 'M247, GB']},
 'sk': {'unique_as_number': ['11325'], 'unique_as_name': ['AS11325, US']},
 'hu': {'unique_as_number': ['9009'], 'unique_as_name': ['M247, GB']}}

In [16]:
os.chdir(output_path_s)

In [17]:
with open('country_as.json', 'w', encoding='utf-8') as f:
    json.dump(as_d, f, ensure_ascii=False, indent=4)